# Automatic Error Handling

Sometimes there are parts of the design space that we want to explore that will cause the EnergyPlus simulation to fail, such as invalid combinations of parameter values. 

In [1]:
import pandas as pd
import copy
from subprocess import CalledProcessError

from besos import eppy_funcs as ef
from besos import sampling
from besos.evaluator import EvaluatorEP
from besos.parameters import RangeParameter, CategoryParameter, Parameter, FieldSelector
from besos.problem import EPProblem

In this example, we load the emaple model and try to use an undefined material `Invalid Material`.

In [2]:
building = ef.get_building()

problem = EPProblem([
    Parameter(
        FieldSelector(object_name='Mass NonRes Wall Insulation', field_name='Thickness'),
        RangeParameter(min_val = 0.01, max_val=0.99)),
    Parameter(
        FieldSelector(class_name='Construction', object_name='ext-slab', field_name='Outside Layer'),
        CategoryParameter(options=('HW CONCRETE', 'Invalid Material')))])

samples = sampling.dist_sampler(sampling.lhs, problem, 5)
samples

,Thickness,Outside Layer
0,0.823552,Invalid Material
1,0.729157,HW CONCRETE
2,0.404515,Invalid Material
3,0.293819,Invalid Material
4,0.065249,HW CONCRETE


By default, evaluation of a DataFrame of parameters will end when an invalid combination is encountered.

In [3]:
try:
    EvaluatorEP(problem, building, error_mode='Failfast', out_dir='outputdir', err_dir='outputdir').df_apply(samples)
except Exception as e:
    print('caught', e)

Program Version,EnergyPlus, Version 9.0.1-bb7ca4f0da, YMD=2019.10.28 15:51,

   ************* Beginning Zone Sizing Calculations

   ** Severe  ** Did not find matching material for Construction EXT-SLAB, missing material = INVALID MATERIAL

   **  Fatal  ** GetSurfaceData: Errors discovered, program terminates.

   ...Summary of Errors that led to program termination:

   ..... Reference severe error count=1

   ..... Last severe error=Did not find matching material for Construction EXT-SLAB, missing material = INVALID MATERIAL

   ************* Warning:  Node connection errors not checked - most system input has not been read (see previous warning).

   ************* Fatal error -- final processing.  Program exited before simulations began.  See previous error messages.

   ************* EnergyPlus Warmup Error Summary. During Warmup: 0 Warning; 0 Severe Errors.

   ************* EnergyPlus Sizing Error Summary. During Sizing: 0 Warning; 1 Severe Errors.

   ************* EnergyPlus 

/home/evan/anaconda3/lib/python3.6/site-packages/besos/evaluator.py:222: UserWarning: for inputs: ['Thickness', 'Outside Layer'] problematic values were: (0.8235515577474863, 'Invalid Material')
  warn(msg)


However sometimes we want to have a fallback value for these invalid states.  
For example, with an optimization algorithm that is minimizing, we can set a very high output value.  
This can be specified with the `error_value` argument for evaluators.  
It must be of the form `(objective_values, constraint_values)` where objective_values and constraint_values are tuples of the same length as the number of objectives/constraints.  
Since we have 1 objective and no constraints, we use a tuple with one item for objective_values, and an empty tuple for the constraint values.

In [4]:
error_value=((10.0**20,), ())

EvaluatorEP(problem, building, error_mode='Print', error_value=error_value).df_apply(samples)

Program Version,EnergyPlus, Version 9.0.1-bb7ca4f0da, YMD=2019.10.28 15:51,

   ************* Beginning Zone Sizing Calculations

   ** Severe  ** Did not find matching material for Construction EXT-SLAB, missing material = INVALID MATERIAL

   **  Fatal  ** GetSurfaceData: Errors discovered, program terminates.

   ...Summary of Errors that led to program termination:

   ..... Reference severe error count=1

   ..... Last severe error=Did not find matching material for Construction EXT-SLAB, missing material = INVALID MATERIAL

   ************* Warning:  Node connection errors not checked - most system input has not been read (see previous warning).

   ************* Fatal error -- final processing.  Program exited before simulations began.  See previous error messages.

   ************* EnergyPlus Warmup Error Summary. During Warmup: 0 Warning; 0 Severe Errors.

   ************* EnergyPlus Sizing Error Summary. During Sizing: 0 Warning; 1 Severe Errors.

   ************* EnergyPlus 

Program Version,EnergyPlus, Version 9.0.1-bb7ca4f0da, YMD=2019.10.28 15:51,

   ************* Beginning Zone Sizing Calculations

   ** Severe  ** Did not find matching material for Construction EXT-SLAB, missing material = INVALID MATERIAL

   **  Fatal  ** GetSurfaceData: Errors discovered, program terminates.

   ...Summary of Errors that led to program termination:

   ..... Reference severe error count=1

   ..... Last severe error=Did not find matching material for Construction EXT-SLAB, missing material = INVALID MATERIAL

   ************* Warning:  Node connection errors not checked - most system input has not been read (see previous warning).

   ************* Fatal error -- final processing.  Program exited before simulations began.  See previous error messages.

   ************* EnergyPlus Warmup Error Summary. During Warmup: 0 Warning; 0 Severe Errors.

   ************* EnergyPlus Sizing Error Summary. During Sizing: 0 Warning; 1 Severe Errors.

   ************* EnergyPlus 

/home/evan/anaconda3/lib/python3.6/site-packages/besos/evaluator.py:222: UserWarning: for inputs: ['Thickness', 'Outside Layer'] problematic values were: (0.4045145306107555, 'Invalid Material')
  warn(msg)


Program Version,EnergyPlus, Version 9.0.1-bb7ca4f0da, YMD=2019.10.28 15:51,

   ************* Beginning Zone Sizing Calculations

   ** Severe  ** Did not find matching material for Construction EXT-SLAB, missing material = INVALID MATERIAL

   **  Fatal  ** GetSurfaceData: Errors discovered, program terminates.

   ...Summary of Errors that led to program termination:

   ..... Reference severe error count=1

   ..... Last severe error=Did not find matching material for Construction EXT-SLAB, missing material = INVALID MATERIAL

   ************* Warning:  Node connection errors not checked - most system input has not been read (see previous warning).

   ************* Fatal error -- final processing.  Program exited before simulations began.  See previous error messages.

   ************* EnergyPlus Warmup Error Summary. During Warmup: 0 Warning; 0 Severe Errors.

   ************* EnergyPlus Sizing Error Summary. During Sizing: 0 Warning; 1 Severe Errors.

   ************* EnergyPlus 

/home/evan/anaconda3/lib/python3.6/site-packages/besos/evaluator.py:222: UserWarning: for inputs: ['Thickness', 'Outside Layer'] problematic values were: (0.2938185752328907, 'Invalid Material')
  warn(msg)


,Electricity:Facility
0,1.000000e+20
1,1.814138e+09
2,1.000000e+20
3,1.000000e+20
4,1.847449e+09


This time, we got a warning for the invalid states, and our error value was used as the result.  
If we do not want to display these warnings, we can set the `error_mode='Silent'`.  
Omiting the error value will use a reasonable default, set to the opposite of what we are optimizing each objective towards. (This does not work for problems with constraints).

In [5]:
evaluator = EvaluatorEP(problem, building, error_mode='Silent')
print('Error value defaulted to:', evaluator.error_value)

Error value defaulted to: ((inf,), ())


In [6]:
evaluator.df_apply(samples)

,Electricity:Facility
0,inf
1,1.814138e+09
2,inf
3,inf
4,1.847449e+09
